In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:skyblue;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Tabular Playground Series - Mar 2022 : Random Forest + GridSearch
</h1>
</div>

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/25226/logos/header.png?t=2021-01-27-17-34-31" alt="">

In [ ]:
import pandas as pd

df_train = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv')
df_train

In [ ]:
df_test = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv')
df_test

In [ ]:
df_train['time'] = df_train['time'].apply(pd.to_datetime)

df_train['year'] = df_train['time'].apply(lambda x : x.year)
df_train['month'] = df_train['time'].apply(lambda x : x.month)
df_train['day'] = df_train['time'].apply(lambda x : x.day)
df_train['hour'] = df_train['time'].apply(lambda x : x.hour)
df_train.drop(['time', 'year', 'month', 'day'], axis=1, inplace=True)
df_train.head()

In [ ]:
df_test['time'] = df_test['time'].apply(pd.to_datetime)

df_test['year'] = df_test['time'].apply(lambda x : x.year)
df_test['month'] = df_test['time'].apply(lambda x : x.month)
df_test['day'] = df_test['time'].apply(lambda x : x.day)
df_test['hour'] = df_test['time'].apply(lambda x : x.hour)
df_test.drop(['time', 'year', 'month', 'day'], axis=1, inplace=True)
df_test.head()

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:skyblue;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Simple EDA
</h1>
</div>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(figsize=(16, 8))
mean, std = df_train['congestion'].mean(), df_train['congestion'].std()
normal_dist = np.random.normal(mean, std, len(df_train))
sns.kdeplot(data=df_train, x='congestion', ax=ax, label='congestion')
sns.kdeplot(x=normal_dist, ax=ax, label='normal distribution')
ax.set_title('KDE Plot')
plt.legend();

In [ ]:
sns.catplot(x="hour", y="congestion", data=df_train, kind="violin")
plt.show()

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:skyblue;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Data Preprocessing
</h1>
</div>

In [ ]:
x_train = df_train.drop(['congestion','row_id'], axis=1 )
y_train = df_train['congestion']
x_test = df_test.drop(['row_id'], axis=1 )

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
x_train['direction'] = le.fit_transform(x_train['direction'])
x_test['direction'] = le.transform(x_test['direction'])

<div class="text_cell_render border-box-sizing rendered_html">
<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:skyblue;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
<h1 style="text-align: center;
           padding: 10px;
              color:white">
Random Forest Regressor + GridSearchCV
</h1>
</div>

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'criterion':['squared_error','friedman_mse'], 'max_depth':[None,2,5,10,20,50], 'max_leaf_nodes':[None,2,5,10,20,50]}
]

reg = RandomForestRegressor()
grid_search = GridSearchCV(reg, param_grid, cv=5, scoring = 'neg_mean_squared_error', verbose=2)
grid_search.fit(x_train, y_train)

In [ ]:
print('GridSearchCV Best Params:', grid_search.best_params_)

In [ ]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)

In [ ]:
df_submit = pd.read_csv('../input/tabular-playground-series-mar-2022/sample_submission.csv')
df_submit['congestion'] = y_pred
df_submit.to_csv('submission.csv', index=False, header=True)